# 0. Load Libraries

In [1]:
import os
import json
import math
import random
import pandas as pd

from models import Model_Multiperiod
from classes import  Locatable, Satellite, Segment, Vehicle
from drawingmap import DrawingMap

# 1. Functions

## 1.1. Load Satellites

In [2]:
def load_satellites(DEBUG = False) -> dict[str, Satellite]:
    satellites = {}
    df = pd.read_csv('../others/Levantamiento de Información/base_satellites.csv')
    for i in range(len(df)):
        id_s = str(df.nombre[i])
        new_satellite = Satellite(id=id_s
                               , lon=df.longitud[i]
                               , lat=df.latitud[i]
                               , distanceFromDC=df.loc[i, 'distance.value']/1000
                               , durationFromDC=df.loc[i, 'duration.value']/3600
                               , durationInTrafficFromDC=df.loc[i, 'duration_in_traffic.value']/3600
                               , costFixed=json.loads(df.loc[i, 'costFixed'])
                               , capacity=json.loads(df.loc[i, 'capacity'])
                               )
        satellites[id_s] = new_satellite
    if DEBUG:
        print("-"*50)
        print("Count of SATELLITES: ", len(satellites))
        print("First Satellite:")
        print(json.dumps(list(satellites.values())[0].__dict__,indent=2,default=str))
    return satellites, df

## 1.2. Load Customer Segments

In [18]:
def load_customer_segments(DEBUG = False) -> dict[str, Segment]:
    segments = {}
    df = pd.read_csv('../others/Levantamiento de Información/Caracterización de hexagonos de La Paz.csv')

    # filtered only rows with data cajas > 0
    df = df[df['suma.cajas'] > 0].reset_index(drop=True).copy()
    #

    for i in range(len(df)):
        id_k = str(df.loc[i, 'h3_address'])
        new_segment = Segment(id=id_k
                              ,lon=df.loc[i, 'lon.centroid']
                              ,lat=df.loc[i, 'lat.centroid']
                              ,areaKm=df.loc[i, 'area.km2']
                              ,avgTickets=df.loc[i, 'media.boletas.mes']
                              ,avgCustomers=df.loc[i ,'media.clientes.mes']
                              ,avgPackages=df.loc[i, 'media.suma.cajas.mes']
                              ,avgPackagesBySales=df.loc[i, 'media.cajas.por.venta']
                              ,packages=df.loc[i, 'suma.cajas']
                              ,sales=df.loc[i, 'Cantidad.ventas']
                              ,customers=df.loc[i, 'Total.clientes']
                              ,avgDropSize=df.loc[i, 'media.cajas.cliente']
                              )
        segments[id_k] = new_segment
    if DEBUG:
        print("-"*50)
        print("Count of segments: ", len(segments))
        print("First segment:")
        print(json.dumps(list(segments.values())[0].__dict__,indent=2,default=str))
    return segments, df

## 1.3. Load Vehicles

In [19]:
small_vehicle = Vehicle(id='small'
                        , capacity=20
                        , costFixed=10)
large_vehicle = Vehicle(id='large'
                        , capacity=120
                        , costFixed=50)

## 1.4. Load Matrixes: Distance and Durations with and without traffic

### 1.4.1. From Satellites to Clusters

In [20]:
def load_distances_duration_matrix_from_satellite(DEBUG = False) -> dict[str, dict]:
    df = pd.read_csv('../others/Levantamiento de Información/Informacion Satelites a Hexagonos.csv')
    size = len(df)
    distance = dict(
        [((df.Satelite[i], df.h3_address[i]), df.loc[i,"distance.value"]/1000) for i in range(size)]
    )
    duration = dict(
        [((df.Satelite[i], df.h3_address[i]), df.loc[i,"duration.value"]/3600) for i in range(size)]
    )
    duration_in_traffic = dict(
        [((df.Satelite[i], df.h3_address[i]), df.loc[i,"duration_in_traffic.value"]/3600) for i in range(size)]
    )
    matrixes = {
        'duration':duration,
        'distance':distance,
        'duration_in_traffic':duration_in_traffic
    }
    return matrixes

### 1.4.2. From DC to Clusters

In [25]:
def load_distances_duration_matrix_from_dc(DEBUG = False) -> dict[str, dict]:
    df = pd.read_csv('../others/Levantamiento de Información/distance_from_dc_to_clusters.csv')
    size = len(df)
    distance = dict(
        [(df.h3_address[i], df.loc[i,"distance"]/1000) for i in range(size)]
    )
    duration = dict(
        [(df.h3_address[i], df.loc[i,"duration"]/3600) for i in range(size)]
    )
    duration_in_traffic = dict(
        [(df.h3_address[i], df.loc[i,"duration_in_traffic"]/3600) for i in range(size)]
    )
    matrixes = {
        'duration':duration,
        'distance':distance,
        'duration_in_traffic':duration_in_traffic
    }
    return matrixes

## 1.5. Arce Function - Fleet Size

# 2. Drawing Map

In [29]:
segments, df_segments = load_customer_segments()
satellites, df_satellites = load_satellites()

## 2.1. La Paz with customer segments

In [30]:
location_la_paz = (-16.501457, -68.149887)
location_DC = (-16.5354544, -68.1958506)

drawer = DrawingMap(location_la_paz)

drawer.addNodes(list_locatables=list(segments.values()), color="blue", radius=1)
drawer.addNodes(list_locatables=list(satellites.values()), color='green', radius=3)
drawer.addNode(location=location_DC, radius=5, color='red')

map = drawer.viewMap()
map

# 3. Model

## 3.1. Params

## 3.2. Model Deterministic

In [ ]:
model_deterministic = Model_Multiperiod(periods=periods)

In [ ]:
model_deterministic.build(

)

In [ ]:
print(model_deterministic.optimizeModel())

In [ ]:
model_deterministic.model.getObjective().getValue()